In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

<div dir="rtl" lang="he" xml:lang="he">

# מודלים סידרתיים

מודלים סידרתיים הם רשתות נורונים שיכולים לתת פרדיקציה על סדרת נתונים באורך משתנה. 
לדוגמה אם נתון לנו ביצועי חברות לאורך זמן, נרצה לבנות מודל שבהתנן ביצועים לאורך תקופה מסויימת יתן לנו תחזית על הביצועים הבאים של החברה. 



<div dir="rtl" lang="he" xml:lang="he">

## שיכבת LSTM
שיכבת 
_LSTM_ - 
long short-term memory
היא מבנה ספציפי של רשת נויירונים שמאפשרת להשתמש במידע סידרתי. 
היא בנוייה מיחידת חישוב המכילה 3 קלטים: 
1. הקלט הנוכחי
$x_t$. 
זה הקלט של הזה, אם לדוגמה יש לנו סדרת נתונים באורך 
$T$
אזי בשלב 
$t$
הקלט הוא
$x_t$
2. הזיכרון קצר הטווח
$S_{t-1}$. 
_short term memory_
זהו הזיכרון של היחידה הקודמת. 
3. הזיכרון ארוך הטווח
$L_{t-1}$. 
_long term memory_
זה חישוב מסויים של כל הקלטים הקודמים. 

בהתאמה כל יחידה פולטת 2 פלטים:
1. הזיכרון קצר הטווח
$S_{t-1}$. 
_short term memory_
זהו הזיכרון של היחידה הקודמת. 
כאשר מגיעים לשלב האחרון, הוא מהווה הפלט הסופי של היחידה, הפרדיקציה.
2. הזיכרון ארוך הטווח
$L_{t-1}$.
זה חישוב מסויים של כל הקלטים הקודמים.


בתהליך חישוב היחידה נשתמש הרבה פעמים בשיכבה לינארית מהצורה הבאה 
$Lin: (x_1,x_2)\to y$
המוגדרת ע"י 
$y=Lin(x_1,x_2)=\sigma(W_1x_1+W_2x_2+b)$.
כאשר 
$\sigma$
היא פונקציית אקטיבציה כל שהיא.

#### מידת הזיכרון לזיכרון לטווח ארוך - _forget gate_
בשלב זה נחליט על המקדם של הזיכרון לטווח הארוך. 
נחשב זאת ע"י 
$F_t=\sigma_g(Lin(x_t,S_{t-1}))\cdot L_{t-1}$.
נרצה שתמיד המקדם יהיה בן 
$0-1$
#### מקדם העידכון _update gate's activation vector_
נתון ע"י 
$$ 
U_t=\sigma_g(Lin(x_t,S_{t-1}))
$$
#### מקדם הפלט _output gate's activation vector_
נתון ע"י
$$
O_t=\sigma_g(Lin(x_t,S_{t-1}))
$$
#### פלט היחידה  _output vector of the LSTM unit_
נתון ע"י
$$
S_t=U_t\cdot L_{t-1}+F_t\cdot S_{t-1}
$$




<div dir="rtl" lang="he" xml:lang="he">

### החישוב 

<div dir="rtl" lang="he" xml:lang="he">

נסתכל על שווי כל כמה מטבעות וירטואלים לאורך זמן

[data](https://www.kaggle.com/datasets/joebeachcapital/crypto-currencies/download?datasetVersionNumber=1)


In [113]:
df_btc = pd.read_csv("dataset/cryptocurrency-historical-prices/coin_Bitcoin.csv")
df_eth = pd.read_csv("dataset/cryptocurrency-historical-prices/coin_Ethereum.csv")
df_eos = pd.read_csv("dataset/cryptocurrency-historical-prices/coin_EOS.csv")

In [114]:
df_btc.merge(df_eth, how='outer').merge(df_eos, how='outer').to_csv("dataset/cryptocurrency-historical-prices/merged.csv", index=False)

In [115]:
df = pd.read_csv("dataset/cryptocurrency-historical-prices/merged.csv")

In [118]:
grouped_df = df.groupby("Date").count().reset_index()
start_date = grouped_df[grouped_df['Close'] == 3].min(axis=0)['Date']
end_date = df['Date'].max()
df = df.where((df['Date'] >= start_date) & (df['Date'] <= end_date)).dropna()

In [121]:
df

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
1525,1526.0,Bitcoin,BTC,2017-07-02 23:59:59,2514.280029,2394.840088,2436.399902,2506.469971,8.037470e+08,4.116702e+10
1526,1527.0,Bitcoin,BTC,2017-07-03 23:59:59,2595.000000,2480.469971,2498.560059,2564.060059,9.641120e+08,4.211821e+10
1527,1528.0,Bitcoin,BTC,2017-07-04 23:59:59,2631.590088,2559.350098,2561.000000,2601.639893,9.855160e+08,4.274062e+10
1528,1529.0,Bitcoin,BTC,2017-07-05 23:59:59,2622.649902,2538.550049,2602.870117,2601.989990,9.415660e+08,4.275109e+10
1529,1530.0,Bitcoin,BTC,2017-07-06 23:59:59,2616.719971,2581.689941,2608.100098,2608.560059,7.619570e+08,4.286376e+10
...,...,...,...,...,...,...,...,...,...,...
6612,1462.0,EOS,EOS,2021-07-02 23:59:59,4.000508,3.751986,3.930561,3.934325,1.014258e+09,3.756672e+09
6613,1463.0,EOS,EOS,2021-07-03 23:59:59,4.060837,3.859586,3.926595,4.046421,7.302703e+08,3.863820e+09
6614,1464.0,EOS,EOS,2021-07-04 23:59:59,4.181861,3.952532,4.048645,4.059930,8.086025e+08,3.876834e+09
6615,1465.0,EOS,EOS,2021-07-05 23:59:59,4.061888,3.773130,4.061888,3.819266,8.004033e+08,3.647131e+09


In [123]:
px.line(df, x='Date', y='Close', color='Symbol')

<div dir="rtl" lang="he" xml:lang="he">

> בהערה יש לומר, שהדאטה הזה לא טוב ללמידה, הוא מכיל סדרות נתונים שונות מידיי שיהיה קשה לבנות מודל שיסיק מהם מסקנות משמעותיות

In [ ]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=1, num_layers=1, batch_first=True)
        self.linear = nn.Linear(1, 1)